<a href="https://colab.research.google.com/github/carlos-alves-one/-SSDM-Coursework-1/blob/main/glaucoma_detection_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Artificial Intelligence
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework No.2

#Project
VisionGuard AI: Deep Learning for Early Glaucoma Detection

# Introduction

The purpose of this research is to outline the progression of a deep-learning model designed to identify glaucoma through the analysis of ocular pictures. Glaucoma is a debilitating ocular disorder that, if left undetected and untreated in its early stages, can result in complete vision loss. Effective screening procedures are necessary due to the asymptomatic nature of the early stages of glaucoma. Deep learning, specifically convolutional neural networks (CNNs), has demonstrated considerable potential in image identification tasks and can aid in the early detection of glaucoma. The dataset utilised in this research comprises a collection of ocular pictures accompanied by a binary classification showing the presence or absence of glaucoma. The ExpCDR, or 'Cup to Disc Ratio', is a crucial clinical parameter utilised in evaluating glaucoma for each image.

# Methodology

## Data Preprocessing

The photos will undergo a process of loading, resizing to a consistent dimension, and normalisation to ensure that their pixel values fall within the range of 0 to 1. Furthermore, it is possible to employ data augmentation methods, such as rotations, shifts, and flips, in order to augment the size and diversity of the dataset. This can be beneficial in mitigating the issue of overfitting.

### Load the data

In [1]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import the pandas library and give it the alias 'pd' for data manipulation and analysis
import pandas as pd

# Load the dataset glaucoma from Google Drive
data_path = '/content/drive/MyDrive/glaucoma.csv'
glaucoma_data = pd.read_csv(data_path)

# Display the first few rows of the dataframe
glaucoma_data.head()


,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


Dataset source: https://www.kaggle.com/datasets/sshikamaru/glaucoma-detection

License: CC0 - Public Domain
https://creativecommons.org/publicdomain/zero/1.0/

The dataset contains the following columns:

    - Filename: The name of the image file.
    - ExpCDR: The 'Cup to Disc Ratio', a crucial parameter for evaluating glaucoma.
    - Eye: Indicates which eye the image corresponds to (OD for right eye, OS for left eye).
    - Set: This could denote the dataset split (e.g., training, validation, or test set), but we would need further clarification.
    - Glaucoma: The binary label indicating the presence (1) or absence (0) of glaucoma.

###Preprocess the Data

Declare function to preprocess a single image:

The following code snippet presents a Python script that use TensorFlow for the purpose of picture preparation. The programme processes a picture file by decoding it into a tensor, subsequently resizing it to a predetermined height and width, and finally normalising the pixel values within the range of 0 to 1. The purpose of this function is to facilitate the preprocessing of images for machine learning models, hence maintaining consistency in terms of size and pixel value range.

In [3]:
# Importing the os module for interacting with the operating system and tensorflow for machine learning tasks
import os
import tensorflow as tf

# Function to preprocess a single image
def preprocess_image(filename, img_height=224, img_width=224, images_directory='/content/drive/MyDrive/Images'):

    # Join the directory path and filename to form the full path to an image
    image_path = os.path.join(images_directory, filename)

    # Read the image file from the specified path into a tensor
    image = tf.io.read_file(image_path)

    # Decode the JPEG image and ensure it has 3 color channels (RGB)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize the image to the specified height and width using TensorFlow's resize function
    image = tf.image.resize(image, [img_height, img_width])

    # Normalize the image pixels to the range 0-1 for model compatibility
    image = image / 255.0

    # Return image preprocessed
    return image


###Data Augmentation

 Set up data augmentation using the ImageDataGenerator class from tf.keras.preprocessing.image:

 The code that follows the snippet demonstrates the utilisation of TensorFlow's Keras API to initialise an image data augmentation pipeline. More specifically, it employs the ImageDataGenerator class. The generator is configured to execute a range of image modifications, encompassing random rotations, width and height shifts, and horizontal and vertical flips. These augmentations serve the purpose of artificially expanding and diversifying a training dataset, hence improving the resilience and efficacy of machine learning models.

In [4]:
# Import the ImageDataGenerator class from TensorFlow's Keras API for real-time data augmentation of images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation
data_augmentation = ImageDataGenerator(

    # Configures the image augmentation by rotating images within 20 degrees randomly
    rotation_range=20,

    # Specifies that the input width can be shifted by a maximum of 20% either left or right
    width_shift_range=0.2,

    # Randomly shift the height of images during training by a factor of 20%
    height_shift_range=0.2,

    # Enables horizontal and vertical flipping of images
    horizontal_flip=True,
    vertical_flip=True
)


###Apply Preprocessing and Augmentation to Dataset

The following code snippet is responsible for extracting filenames and their matching glaucoma presence labels from a dataset. Additionally, it establishes a directory path for the picture files. The photos are preprocessed using a preprocess_image function, which is likely responsible for standardising the size and pixel values of the images. Ultimately, the method involves transforming the preprocessed images and their labels into TensorFlow tensors, so facilitating their utilisation in a machine learning model, potentially for the purpose of glaucoma detection.

In [5]:
# Extract filenames and corresponding glaucoma presence labels from the dataset
filenames = glaucoma_data['Filename'].values
labels = glaucoma_data['Glaucoma'].values

# A placeholder for the images directory
images_directory = '/content/drive/MyDrive/Images'

# Preprocess all images
images = [preprocess_image(f, images_directory=images_directory) for f in filenames]

# Convert to Tensor
images = tf.stack(images)
labels = tf.convert_to_tensor(labels)


###Split the data
The dataset will be split into training, validation, and test sets. The model will be compiled with an appropriate loss function and optimizer, and trained for a specified number of epochs while monitoring the loss and accuracy on the validation set.

In [6]:
# Import train_test_split function from scikit-learn to split data into training and test sets
from sklearn.model_selection import train_test_split

# Import TensorFlow for deep learning and train_test_split function for splitting the dataset into training and testing sets
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Convert image and label tensors to numpy arrays for further manipulation or analysis
images_numpy = images.numpy()
labels_numpy = labels.numpy()

# Split the dataset into training set and a combined validation/test set with a 20% size of the original dataset,
# using a fixed random state for reproducibility.
train_images, val_test_images, train_labels, val_test_labels = train_test_split(
    images_numpy, labels_numpy, test_size=0.2, random_state=42
)

# Split the val_test set equally into validation and test sets (50% validation, 50% test)
val_images, test_images, val_labels, test_labels = train_test_split(
    val_test_images, val_test_labels, test_size=0.5, random_state=42)


## Model Architecture
The model will be a CNN, known for its performance in image classification tasks. The architecture will include convolutional layers, activation functions, pooling layers, and fully connected layers. Dropout layers may be included to reduce overfitting.

The code snippet sets up the label data for a neural network-based classification model, ensuring that the labels are in a format that can be effectively used for training and evaluation within a TensorFlow framework. This preprocessing step is critical to the machine learning pipeline for classification problems.

In [7]:
# Import the NumPy library for numerical operations.
import numpy as np

# Determine the number of unique classes in the training labels
num_classes = len(np.unique(train_labels))

# Convert train labels to one-hot encoded format
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)

# Convert validation labels to one-hot encoded format
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes=num_classes)

# Convert test labels to one-hot encoded format
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)


This code demonstrates the construction, training, and evaluation of a Convolutional Neural Network (CNN) for image classification using TensorFlow. It includes defining the model architecture with convolutional, pooling, and dense layers, followed by compilation with appropriate loss and optimization functions. The model is then trained on labeled image data, evaluated for accuracy on a test set, and the test accuracy is reported, showcasing the end-to-end process of a typical deep learning image classification task.


In [8]:
# Define the input layer with shape 224x224 and 3 color channels (RGB)
inputs = tf.keras.Input(shape=(224, 224, 3))

# Apply a 2D convolution layer with 32 filters, 3x3 kernel, and ReLU activation
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)

# Add a max pooling layer with a 2x2 pool size to reduce spatial dimensions
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)

# Add another convolution layer with 64 filters, 3x3 kernel, and ReLU activation
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)

# Add a second max pooling layer to further reduce dimensions.
x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)

# Add a third convolution layer with 128 filters, 3x3 kernel, and ReLU activation
x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)

# Flatten the output to prepare for the dense layer
x = tf.keras.layers.Flatten()(x)

# Determine the number of classes from the shape of the training labels
num_classes = train_labels.shape[1]

# Create the output layer with 'num_classes' neurons and softmax activation
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

# Build the model by specifying inputs and outputs
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Print the accuracy results for this model
print(f"Test accuracy: {test_accuracy:.4f}")


Epoch 1/10
15/15 [==============================] - 13s 107ms/step - loss: 0.9423 - accuracy: 0.6389 - val_loss: 0.5324 - val_accuracy: 0.8269
Epoch 2/10
15/15 [==============================] - 1s 52ms/step - loss: 0.5819 - accuracy: 0.7329 - val_loss: 0.4994 - val_accuracy: 0.8269
Epoch 3/10
15/15 [==============================] - 1s 52ms/step - loss: 0.5862 - accuracy: 0.7329 - val_loss: 0.4713 - val_accuracy: 0.8269
Epoch 4/10
15/15 [==============================] - 1s 54ms/step - loss: 0.5737 - accuracy: 0.7329 - val_loss: 0.5324 - val_accuracy: 0.8269
Epoch 5/10
15/15 [==============================] - 1s 57ms/step - loss: 0.5670 - accuracy: 0.7350 - val_loss: 0.4842 - val_accuracy: 0.8269
Epoch 6/10
15/15 [==============================] - 1s 57ms/step - loss: 0.5533 - accuracy: 0.7457 - val_loss: 0.4864 - val_accuracy: 0.8846
Epoch 7/10
15/15 [==============================] - 1s 58ms/step - loss: 0.5406 - accuracy: 0.7436 - val_loss: 0.4676 - val_accuracy: 0.8846
Epoch 8/10


###Analysis of the Initial Model

This model is a relatively simple Convolutional Neural Network (CNN) model designed for image classification tasks. It has been built using TensorFlow and Keras, and the architecture is straightforward, making it suitable for small to medium-sized datasets and a starting point for more complex tasks. Here is a breakdown of the model:

1. Input Layer: Accepts images of size 224x224 with three colour channels (RGB).
2. Convolutional Layers:
   - The first convolutional layer has 32 filters of size 3x3 with ReLU activation.
   - The second convolutional layer has 64 filters of size 3x3 with ReLU activation.
   - The third convolutional layer has 128 filters of size 3x3 with ReLU activation.
3. Pooling Layers: Two max-pooling layers are used to reduce the spatial dimensions of the feature maps.
4. Flatten Layer: Flattens the output for the dense layer.
5. Output Layer: A dense layer with some neurons equal to the number of classes (`num_classes`), using softmax activation for multi-class classification.

The model is compiled with the Adam optimizer, categorical cross-entropy loss, and accuracy metric. It has trained for ten epochs with a validation split of 0.1.

This model is suitable for learning or initial experimentation with image classification tasks.